# The following are analysis performed for this Bachelor Thesis

In [1]:
%load_ext autoreload
%autoreload 2

import header as head
import json
import os

annotations_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations"
community_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection"
bridges_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges"

There are 74 rumour source tweet with 0 non-rumour source tweet within the total of 74 rumour tweet.

In [ ]:
# p = head.getPath()
# print(p)
head.count()

In [ ]:
# merging following relationship into a JSON file
head.followRelate()

with open('/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json') as f:
    data = json.load(f)
    followRelate_Ids = list(data.keys())

In [ ]:
# generate dictionary for tweetId and userId conversion
# results are saved in Id-conversions folder
head.gen_idConversion()

There are followers who reacted to the source tweets. 
* reacts from followers are saved in `react_follows`
* reacts from non-followers are saved in `react_not_follows`

In [ ]:
# See whether there are any reactors that follows the person they responded to 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json") as f:
    followingRelate = json.load(f)

head.is_follower_react(followingRelate)

In [12]:
# A bridge has followers from both communities
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as f:
    user_followed_by = json.load(f)

two_com_bridges = {}
for id in user_followed_by:
   user = user_followed_by[id]
   if len(user["rumours"]) > 0 and len(user["non_rumours"]) > 0:
       two_com_bridges[id] = user_followed_by[id]

head.writeToJSON(bridges_path,"both_communities",two_com_bridges)

In [13]:
# A bridge only has followers from one community but follows other community 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as f:
    follows = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as f:
    user_followed_by = json.load(f)

one_com_bridges = {}
popList = []

for id in user_followed_by:
    user = user_followed_by[id]
    if len(user["rumours"]) == 0 or len(user["non_rumours"]) == 0:
        if len(user["rumours"]) == 0 and len(user["non_rumours"]) == 0:
            continue
        
        else : one_com_bridges[id] = user_followed_by[id]

for b in one_com_bridges:
    fans = one_com_bridges[b]
    fans["follow"]=[]
    try: followList = follows[b]
    except: continue

    if len(fans["rumours"]) == 0:
        for f in followList:
            if f in rumours_set: fans["follow"].append(f)
    else:
        for f in followList:
            if f in non_rumours_set: fans["follow"].append(f)

    if (len(fans["follow"])==0): popList.append(b)

for p in popList:
    one_com_bridges.pop(p)

head.writeToJSON(bridges_path,"single_community2",one_com_bridges)

In [9]:
# A bridge only has followers from one community but follows other bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as f:
    followers = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as f:
    follows = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/both_communities.json") as f:
    bi_commun_bridges = json.load(f)
    bi_commun_bridges = list(bi_commun_bridges.keys())

one_com_bridges = {}
for id in followers:
    user = followers[id]
    if len(user["rumours"]) == 0 or len(user["non_rumours"]) == 0:
        if len(user["rumours"]) == 0 and len(user["non_rumours"]) == 0:
            continue
        
        else : one_com_bridges[id] = followers[id]

for b in list(one_com_bridges):
    fans = one_com_bridges[b]
    fans["followed_bridges"]=[]
    try: followList = follows[b]
    except: continue

    for f in followList:
        if f in bi_commun_bridges : fans["followed_bridges"].append(f)
        if f in fans["rumours"] : fans["rumours"].remove(f)
        if f in fans["non_rumours"] : fans["non_rumours"].remove(f)

    if len(fans["followed_bridges"])==0: one_com_bridges.pop(b)

head.writeToJSON(bridges_path,"follows_bridges",one_com_bridges)

In [43]:
#  filter out the bridges ids from following list
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as f:
    following_relate = json.load(f)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/both_communities.json") as f:
    bi_commun_bridges = json.load(f)
    bi_commun_bridges = list(bi_commun_bridges.keys())

new_follow = {}

for id in following_relate:
    user = following_relate[id]
    new_follow[id] = {
        "rumours":[],
        "non_rumours":[],
        "bridges":[]
    }

    rumours = (new_follow[id])["rumours"]
    non_rumours = (new_follow[id])["non_rumours"]
    bridges = (new_follow[id])["bridges"]
    
    
    for r in user["rumours"]:
        if r in bi_commun_bridges: 
            bridges.append(r)
        else:
            rumours.append(r)
    
    for r in user["non_rumours"]:
        if r in bi_commun_bridges: 
            bridges.append(r)
        else:
            non_rumours.append(r)

head.writeToJSON(bridges_path,"followers",new_follow)

In [2]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/both_communities.json") as f:
    bi_commun_bridges = json.load(f)
    bi_commun_bridges = list(bi_commun_bridges.keys())

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as f:
    communities = json.load(f)

communities["bridges"] = []

for id in communities["rumours"]:
    if (id == "428333") : print(id)
    if id in bi_commun_bridges : 
        (communities["rumours"]).remove(id)
        (communities["bridges"]).append(id)

for id in communities["non_rumours"]:
    if id in bi_commun_bridges : 
        (communities["non_rumours"]).remove(id)
        (communities["bridges"]).append(id)

# head.writeToJSON(community_path,"with_bridges3",communities)

## Centrality Analysis

**Bridge Analysis**
- Type 1: Bridges that have followers from both communities
  -  Betweeness Centrality : 
  
- Type 2: Bridges that have followers only from one communities, and follow one or more bridges itself. 
  -  Betweeness Centrality : How critical is the node in spreading information into the community where all its followers are from.
  -  Closeness Centrality : How efficient it is for information to spread from this user/node.

**Community Analysis**
- Closeness Centrality : lower the closeness indicates the more important the node is within its echo chamber. 
-  